# Annotator Helper
This notebook uses a trained model to help us annotate images. 

It'll find the images on the vgg file that have no annotations, load those images, apply the model to them, and then will save it back to the json file for further exploration.


In [8]:
import sys
if '..' not in sys.path:
	sys.path.append('..')

In [15]:
from __future__ import division, print_function

import copy
import json
import os
import time
from typing import Dict, Iterable, List, Tuple, TypedDict, Union, cast

import matplotlib.pyplot as plt
import numpy as np
import reload
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torchvision
import torchvision.models
from PIL import Image, ImageOps
from torch import Tensor, tensor
from torchvision import datasets, transforms

In [20]:
%reload money_counter

from money_counter import constants, data, models, via_utils, engine
from money_counter.data import ViaDatasetOnlyAnnotated, to_via_transform
from money_counter.engine import apply_nms
from money_counter.models import PredictedTarget, Target
from money_counter.utils import decode_data
from vgg_image_annotation import v2
from vgg_image_annotation.v2 import ImageMetadata

In [17]:
# Load model and weights
model, model_name = models.get_fasterrcnn_pretrained()
version_manager = models.VersionManager('../model_state/')

version_manager.load_model(model_name, model)


(49, 0.3290896941936272)

In [18]:
# Load dataset
DATASET_PATH = os.environ['COINS_DATASET_PATH']

# Load data from annotations
via_file = v2.load_via_v2_file(DATASET_PATH)

transform = data.ComposeViaTransform([
    #transforms.Resize((224, 224)),
    to_via_transform(transforms.ToTensor()),
])

def not_annotated(images_metadata: List[v2.ImageMetadata]) -> List[v2.ImageMetadata]:
    return list(filter(lambda image_metadata: not via_utils.is_annotated(image_metadata), images_metadata))


dataset = ViaDatasetOnlyAnnotated(DATASET_PATH, transform=transform)


In [19]:
# Create dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, collate_fn=data.collate_into_lists)

In [13]:
label_map = {label: i for i, label in enumerate(constants.CLASSES)}
label_map_inverted = {v: k for k, v in label_map.items()}

In [21]:
# Run the model for the images in the dataset
device = engine.get_device()

model.to(device)
model.eval()


def to_shape(tensor: Tensor):
    tensor = tensor.numpy()
    return {
        'name': 'rect',
        'x': float(tensor[0]),
        'y': float(tensor[1]),
        'width': float(tensor[2] - tensor[0]),
        'height': float(tensor[3] - tensor[1])
    }


metadata_partial: List[ImageMetadata] = []


@torch.no_grad()
def main():
    for images, targets in dataloader:
        # move images to the device for inference
        images = [image.to(device) for image in images]
        # Move targets to the device for inference
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        # Apply model
        outputs = cast(List[PredictedTarget], model(images))

        # Apply NMS
        apply_nms(outputs)

        # Convert back the outputs to cpu
        outputs = [{k: v.to('cpu') for k, v in cast(
            List[Tuple[str, Tensor]], t.items())} for t in outputs]

        # Decode the filename
        images_id = [t['image_id'] for t in targets]
        images_filename = [
            *decode_data(dataset.filename_map, [i.item() for i in images_id])]

        # Associate to the targets
        outputs = [output | {'filename': filename}
                   for filename, output in zip(images_filename, outputs)]

        # Now convert back the bounding boxes to the original format
        outputs = [{
            'filename': output['filename'],
            'regions': [{
                'shape_attributes': to_shape(box),
                'region_attributes': {
                    "Value": label_map_inverted[int(label.item())],
                    "Edition": "Unknown",
                    "Side": "Unknown"
                }
            } for box, score, label in zip(output['boxes'], output['scores'], output['labels']) if score > 0.35]
        } for output in outputs]

        for output in outputs:
            metadata_partial.append(output)


main()


KeyboardInterrupt: 

In [ ]:
list(filter(lambda x: not x['regions'], metadata_partial))[0]

In [ ]:
# Index the metadata
d = {}
for item in metadata_partial:
	d[item['filename']] = item

metadata_partial = d

In [ ]:
# load the json with the metadata
via = v2.load_via_v2_file(DATASET_PATH)

for metadata in via['_via_img_metadata'].values():
	if metadata['filename'] in metadata_partial:
		metadata['regions'] = metadata_partial[metadata['filename']]['regions']

# Save the new json
with open(f'{DATASET_PATH}/coins_2.json', 'w') as f:
	json.dump(via, f)